In [1]:
import re

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8m.txt", "r", encoding="utf8")
file = open("en1.8m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')
test = text[0]
test

"Greek Christian scribes played a crucial role in the preservation of Aristotle by copying all the extant Greek language manuscripts of the corpus. The first Greek Christians to comment extensively on Aristotle were John Philoponus, Elias, and David in the sixth century, and Stephen of Alexandria in the early seventh century. John Philoponus stands out for having attempted a fundamental critique of Aristotle's views on the eternity of the world, movement, and other elements of Aristotelian thought. After a hiatus of several centuries, formal commentary by Eustratius and Michael of Ephesus reappears in the late eleventh and early twelfth centuries, apparently sponsored by Anna Comnena."

In [42]:
len(text)

69468

In [4]:
#whole_text = whole_text.replace('\n', '.')

In [5]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

test = append_space(test, ",\])>")
test = append_space(test, ".")
test = append_space(test, "\[(<", False)
print(test)

Greek Christian scribes played a crucial role in the preservation of Aristotle by copying all the extant Greek language manuscripts of the corpus . The first Greek Christians to comment extensively on Aristotle were John Philoponus , Elias , and David in the sixth century , and Stephen of Alexandria in the early seventh century . John Philoponus stands out for having attempted a fundamental critique of Aristotle's views on the eternity of the world , movement , and other elements of Aristotelian thought . After a hiatus of several centuries , formal commentary by Eustratius and Michael of Ephesus reappears in the late eleventh and early twelfth centuries , apparently sponsored by Anna Comnena .


In [6]:
# dictionary = dict()

# for i in range(len(text)):
#     text[i] = append_space(text[i], ",\])>")
#     text[i] = append_space(text[i], ".")
#     text[i] = append_space(text[i], "\[(<", False)
    
#     for w in text[i].split():
#         if w.isalpha():
#             try:
#                 dictionary[w.lower()] += 1
#             except KeyError:
#                 dictionary[w.lower()] = 1

In [7]:
# it = 0

# for i in sorted(dictionary, key=dictionary.get, reverse=True): 
#     if it > 100:
#         break

#     print((i, dictionary[i]), end =" ")
#     it += 1

In [8]:
for i in range(len(text)):
    text[i] = append_space(text[i], ",\])>")
    text[i] = append_space(text[i], ".")
    text[i] = append_space(text[i], "\[(<", False)

In [9]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0
#     grams = dict()

    for i in range(len(text_arr)):
        if not text_arr[i].isalpha():
            continue

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                if not text_arr[j].isalpha():
                    break

                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [10]:
def build_n_gram1(text_arr, n_gram):
    cur_gram = []
    count_gram = 0
    grams = dict()

    for i in range(len(text_arr)):
        if not text_arr[i].isalpha():
            continue

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                if not text_arr[j].isalpha():
                    break

                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [11]:
def build_dict1(text, n_s):
    dictionary = []

    for n in range(n_s):
        dictionary.append(build_n_gram1(text.split(), n + 1))
        
    return dictionary

In [12]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

In [13]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [14]:
# print(SCP_f('Greek Christian', big_dict))
# print(SCP_f('Anna Comnena', big_dict))

if re.match(r'^[a-zA-Z\s]*$', "lol s1"):
    print("yes")

In [15]:
def SCP_f_n(dictionary, table, n):
    for s in dictionary[n - 1]:
        table[s] = SCP_f(s, dictionary)

In [60]:
def main(text, max_gram):
    dictionary = build_dict1(text, max_gram)
    print("Dictionary with frequencies created!")
    
    table = dict()
        
    print("Proceeding to calculating glue value")
    print("Glue value calculated for:")

    for i in range(2, max_gram + 1):
        print(i, "grams")
        SCP_f_n(dictionary, table, i)
    
#     print(table)
    print("Glue value calculated, proceeding to Localmax")
    
    RE = []

    xys = dict()

#     for line in text:

    text_arr = text.split()
    
#     if len(text_arr) < 3:
#         continue

    for i in range(len(text_arr) - 2):
        cur_gram = " ".join(text_arr[i:i+3])
        
        if not re.match(r'^[a-zA-Z\s]*$', cur_gram):
            continue
        
#         print('\n')
#         print(cur_gram)
        
        j = i + 4
        n_gram = 1

        while j < len(text_arr) and n_gram < 5:
            next_gram = " ".join(text_arr[i:j])
            
            if not re.match(r'^[a-zA-Z\s]*$', next_gram):
                break
            
            try:
                #maxes[cur_gram]
                xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
            except KeyError:
                x = table[" ".join(cur_gram.split()[:-1])]
                x = max(x, table[" ".join(cur_gram.split()[1:])])
                xys[cur_gram] = [x, table[next_gram]]

            cur_gram = next_gram

            j += 1
            n_gram += 1
            
        cur_gram = " ".join(text_arr[i:i+3])
        j = i - 1
        n_gram = 1
            
        while j >= 0 and n_gram < 5:
            next_gram = " ".join(text_arr[j:i+3])
            
            if not re.match(r'^[a-zA-Z\s]*$', next_gram):
                break
                
#             print(next_gram)
            
            try:
                #maxes[cur_gram]
                xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
            except KeyError:
                x = table[" ".join(cur_gram.split()[:-1])]
                x = max(x, table[" ".join(cur_gram.split()[1:])])
                xys[cur_gram] = [x, table[next_gram]]

            cur_gram = next_gram

            j -= 1
            n_gram += 1
    
    print(xys)
    # what about bi-grams??????
    for g in xys:
        val = (xys[g][0] + xys[g][1]) / 2
        if table[g] > val:
            RE.append(g)
    
    return RE

In [61]:
RE = main(test, 7)
print("Obtained Relevant Expressions:")
RE

Dictionary with frequencies created!
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Glue value calculated, proceeding to Localmax
{'Greek Christian scribes': [1.0, 0.6000000000000001], 'Greek Christian scribes played': [1.0, 0.5], 'Greek Christian scribes played a': [0.6000000000000001, 0.7142857142857142], 'Greek Christian scribes played a crucial': [1.0, 0.75], 'Christian scribes played': [1.0, 0.6000000000000001], 'Christian scribes played a': [1.0, 1.0], 'Christian scribes played a crucial': [1.0, 1.0], 'Christian scribes played a crucial role': [1.0, 0.75], 'scribes played a': [1.0, 1.0], 'scribes played a crucial': [1.0, 1.0], 'scribes played a crucial role': [1.0, 1.0], 'scribes played a crucial role in': [1.0, 0.6666666666666666], 'played a crucial': [0.3333333333333333, 1.0], 'played a crucial role': [1.0, 1.0], 'played a crucial role in': [1.0, 0.625], 'played a crucial role in the': [0.5714285714285714, 1.0], '

['Christian scribes played',
 'Christian scribes played a crucial role',
 'played a crucial',
 'crucial role in the preservation',
 'role in the preservation',
 'role in the preservation of Aristotle',
 'preservation of Aristotle',
 'preservation of Aristotle by copying',
 'preservation of Aristotle by copying all',
 'Aristotle by copying',
 'Aristotle by copying all',
 'Aristotle by copying all the extant',
 'copying all the extant',
 'all the extant',
 'all the extant Greek language',
 'extant Greek language',
 'extant Greek language manuscripts',
 'manuscripts of the corpus',
 'language manuscripts of the corpus',
 'first Greek Christians',
 'Christians to comment extensively',
 'extensively on Aristotle were',
 'and David in',
 'David in the sixth',
 'David in the sixth century',
 'Stephen of Alexandria',
 'Stephen of Alexandria in the early',
 'Alexandria in the early',
 'Alexandria in the early seventh',
 'early seventh century',
 'Alexandria in the early seventh century',
 'stan

In [18]:
table[1]

{'Greek Christian scribes': 0.009009009009009009,
 'Christian scribes played': 0.0014992503748125937,
 'scribes played a': 5.5320443669958234e-05,
 'played a crucial': 0.0011758557616932323,
 'a crucial role': 0.00022490913670876965,
 'crucial role in': 0.00017957963997873778,
 'role in the': 0.000357817174648488,
 'in the preservation': 8.867478853834021e-06,
 'the preservation of': 6.13510752974209e-05,
 'preservation of Aristotle': 0.006578947368421052,
 'of Aristotle by': 1.7786631567713707e-05,
 'Aristotle by copying': 0.125,
 'by copying all': 0.00014005602240896358,
 'copying all the': 1.768456049446031e-05,
 'all the extant': 0.00019825535289452815,
 'the extant Greek': 1.7664567527225514e-05,
 'extant Greek language': 0.00554016620498615,
 'Greek language manuscripts': 0.006779661016949152,
 'language manuscripts of': 3.1633556877135266e-05,
 'manuscripts of the': 1.1665514697090328e-05,
 'of the corpus': 1.0432968179447053e-05,
 'The first Greek': 2.1369119485431604e-05,
 'fi